# **Python Scoping: Global vs Local**

### **Learning Objectives**

By the end of this lesson, you’ll be able to:

- Distinguish between global, local, and enclosing scopes.
- Understand how Python resolves variable names (LEGB Rule).
- Modify global variables safely.
- Predict behavior with mutable and immutable types in scope.
- Use `nonlocal` and `global` effectively.


### 1. Understanding Namespaces and Scope

In Python, a **namespace** is a mapping between **names** and **objects**. A **scope** defines the region in which this mapping is valid.

Every Python execution context (script, function, class, module) creates a **scope**.


### 2. LEGB Rule: How Python Resolves Names

Python searches for variable names in the following order:

| Level      | Description                                  |
|------------|----------------------------------------------|
| **L**ocal   | Names assigned within a function (inner scope) |
| **E**nclosing | Names in outer function (useful for closures) |
| **G**lobal  | Names defined at the top-level of the module |
| **B**uilt-in| Names pre-defined in Python (e.g., `len`, `sum`) |

Python walks up the **scope chain**—from local to global—when it tries to resolve a name.

## Example: Full LEGB Rule in Action

In [7]:
x = "global variable"

def outer():
    x = "enclosing variable"
    
    def inner():
        x = "local variable"
        print("Inner:", x)
    
    inner()
    print("Outer:", x)

In [8]:
outer()

Inner: local variable
Outer: enclosing variable


In [9]:
print("Global:", x)

Global: global variable


## `global` vs `nonlocal` Keywords

### `global` — Access & modify a global variable inside a function

In [11]:
counter = 0  # Global variable

def outer_function():
    global counter
    counter += 1
    print("Inside outer_function, counter:", counter)

In [12]:
print("Before calling outer_function:", counter)


Before calling outer_function: 0


In [13]:
outer_function()

Inside outer_function, counter: 1


In [14]:
print("After calling outer_function:", counter)

After calling outer_function: 1


### `nonlocal` — Access & modify variables in **enclosing** (non-global) scope


In [15]:
def outer_function():
    counter = 0  # This variable is local to outer_function

    def inner_function():
        nonlocal counter  # refers to counter in the enclosing function
        counter += 1
        print("Inner counter:", counter)

    print("Before inner_function:", counter)
    inner_function()
    print("After inner_function:", counter)


In [16]:
outer_function()

Before inner_function: 0
Inner counter: 1
After inner_function: 1


## Mutable vs Immutable Variables and Scope in Python

Understanding the **difference between mutable and immutable variables** is crucial when dealing with **variable scope**, especially in **functions** and **class-based models**.

### Core Idea:

- **Immutable objects** (like `int`, `float`, `str`, `tuple`) **cannot be changed in place**. Assigning to them inside a function **creates a new local variable** unless `global` or `nonlocal` is used.
- **Mutable objects** (like `list`, `dict`, `set`) **can be modified in place**, even within a local scope.

### Immutable Example: The `int` Pitfall

In [19]:
x = 5  # Global variable (immutable)

def change_x():
    x = x + 1   # UnboundLocalError: local variable 'x' referenced before assignment
    print(x)

In [20]:
change_x()

UnboundLocalError: cannot access local variable 'x' where it is not associated with a value

#### What Went Wrong?

- Python assumes `x` is local because you assign to it inside the function.
- But you're also trying to read the global `x` before assigning it → error.

---

### Fix It with `global`

In [21]:
x = 5  # Global variable

def change_x_fixed():
    global x
    x = x + 1
    print("Fixed x:", x)

In [22]:
change_x_fixed()

Fixed x: 6


In [23]:
x = 5

def foo():
    global x
    print("Before:", x)
    x += 1
    print("After:", x)

foo()

Before: 5
After: 6


### With `nonlocal` (inside nested function)

In [24]:
def outer():
    x = 10  # Enclosing (non-global) scope

    def inner():
        nonlocal x
        x += 5
        print("Modified in inner():", x)

    inner()
    print("After inner():", x)

In [25]:
outer()

Modified in inner(): 15
After inner(): 15


## Mutable Example: Lists Work Differently


In [26]:
my_list = [1, 2, 3]  # Global mutable object

def modify_list():
    my_list.append(4)
    print("Inside function:", my_list)


In [27]:
modify_list()
print("Outside function:", my_list)

Inside function: [1, 2, 3, 4]
Outside function: [1, 2, 3, 4]



> **Why no `global`?** You're not reassigning `my_list`, just modifying it in place.

---

## Summary: Mutability & Scope
- **`int`, `str`** are **immutable** and require `global` to modify inside a function (e.g., `x = x + 1`).  
- **`list`, `dict`** are **mutable** and can be modified in-place **without** `global` (e.g., `my_list.append(4)`).  
- Use `global` only when **reassigning** a global variable, not when **modifying mutable types**.


# Python Scope in Classes — A Deep Dive

## Objective:
By the end of this lesson, you will:
- Understand Python’s LEGB scope model in class contexts
- Differentiate between instance, class, and global scopes
- Know when to use `global`, `nonlocal`, or `instance references`.
- Avoid common scoping bugs in OOP and ML pipelines


## Scope Inside Classes

### 1. **Instance Variables (`self`)**
An **instance variable** is a variable that is **defined inside a class but assigned within a method (usually `__init__`) using `self`**, and it is **unique to each object (instance) of the class**.

### Key Characteristics:
- Prefixed with `self`, like `self.name`.
- Each object has its **own copy** of instance variables.
- Used to **store data that belongs to a specific object**.

### Example:


In [30]:
class DataScientist:
    def __init__(self, name, specialty):
        self.name = name              # instance variable
        self.specialty = specialty    # instance variable

In [31]:
ds1 = DataScientist("Alice", "NLP")
ds2 = DataScientist("Bob", "Computer Vision")

print(ds1.name)
print(ds2.name) 

Alice
Bob


Each object (`ds1`, `ds2`) has its **own values** for `name` and `specialty`. These values are **stored in instance variables**.

---

### **2. Class Variable – Shared Across All Instances**

A **class variable** is a variable that is **defined directly inside a class (not inside a method)** and is **shared among all instances** of that class.

### Key Characteristics:
- Defined **outside methods**, but **inside the class**.
- Shared by **all instances** of the class.
- Usually used to store **class-wide information or configuration**.

### Example:

In [32]:
class Student:
    school_name = "GAIO Academy"  # class variable

In [33]:
s1 = Student()
s2 = Student()
print(s1.school_name)
print(s2.school_name)

GAIO Academy
GAIO Academy


If you modify via `Student.school_name`, it updates for all. But modifying via `s1.school_name = "New"` will create a new **instance variable**!


In [34]:
class Model:
    framework = "PyTorch"   # class variable

    def __init__(self, name):
        self.name = name    # instance variable

In [35]:
m1 = Model("ResNet")
m2 = Model("BERT")

print(m1.framework)
print(m2.framework)

PyTorch
PyTorch


In [36]:
# Change class variable
Model.framework = "TensorFlow"

print(m1.framework)
print(m2.framework)

TensorFlow
TensorFlow


### 3. **Local Variables in Methods**

Variables created inside methods are **local to that method**:

In [37]:
class Calculator:
    def add(self, a, b):
        result = a + b  # local variable: No need to use self.a or self.b because a and b are only used within the add method (local scope)
        return result

**You can test this by creating an instance of the Calculator class and calling the add() method. Since result is a local variable, it will only exist inside the method and won't be accessible outside. Here's how you test it:**

In [38]:
# Create instance
calc = Calculator()

# Call the method
sum_result = calc.add(10, 5)
print("Result of addition:", sum_result)

Result of addition: 15


You can only access `result` through the `add` function.

---

## Scope Case Study: Class vs Instance vs Global

In [39]:
x = "global"

class Demo:
    x = "class"

    def show(self):
        x = "local"
        print(x)

In [40]:
d = Demo()
d.show() 

local


| Scope Level | Variable Value | Access Rule               |
|-------------|----------------|---------------------------|
| Global      | `"global"`     | Module-level access       |
| Class       | `"class"`      | `Demo.x` or `self.__class__.x` |
| Local       | `"local"`      | Exists only in `show()`   |

---

## Accessing & Modifying Global Variables in a Class

### Avoid this pattern unless you have a valid reason:

In [41]:
counter = 0

class Clicker:
    def click(self):
        global counter
        counter += 1

In [42]:
c = Clicker()
c.click()
print(counter)

1


## `nonlocal` in Nested Functions Inside Classes


In [43]:
class Outer:
    def outer_method(self):
        val = 10
        def inner():
            nonlocal val
            val += 1
            print(val)
        inner()

In [44]:
Outer().outer_method()

11


Used when modifying a variable in the **enclosing (but non-global) scope**.

---

## Advanced Scenario: ML Class Design

In [45]:
class MLModel:
    total_models = 0  # class var

    def __init__(self, name):
        self.name = name       # instance var
        self.parameters = {}   # instance var
        MLModel.total_models += 1  # update class var

    def set_param(self, key, value):
        self.parameters[key] = value

In [ ]:
# Creat and instance of the class
model1 = MLModel("LogReg")

# Class method call
model1.set_param("lr", 0.01)

In [47]:
# Check model1's attributes
print("Model 1 Name:", model1.name)
print("Model 1 Parameters:", model1.parameters)
print("Total Models:", MLModel.total_models)

Model 1 Name: LogReg
Model 1 Parameters: {'lr': 0.01}
Total Models: 1


In [48]:
# Confirm class variable is shared
print("Model 1 Total Models (via class):", model1.total_models)

Model 1 Total Models (via class): 1


| Variable          | Scope Type | Accessed As            |
|-------------------|------------|-------------------------|
| `name`            | Instance   | `self.name`             |
| `parameters`      | Instance   | `self.parameters`       |
| `total_models`    | Class      | `MLModel.total_models`  |

## Final Thoughts

Understanding scope in classes is **key to mastering OOP in Python**, especially when building data pipelines, model classes, or training workflows in ML.

The interplay of **instance, class, and global scope** becomes more critical as your projects scale.